In [ ]:
# cut to piece
import cv2
import os
import math
from os.path import join

tile_size = 384 // 2
scales = [50, 75, 100]
# scales = [50, 60, 70, 80, 90, 100]

# lr_raw_folders = "E:/Dataset/amakano2/720p/"
# hr_raw_folders = "E:/Dataset/amakano2/1440p_escale/"
lr_raw_folders = "E:/Dataset/hamidashi/lr"
hr_raw_folders = "E:/Dataset/hamidashi/hr"
lr_cut_folders = "R:/hamidashi/lr"
hr_cut_folders = "R:/hamidashi/hr"

for folder_path in [hr_cut_folders, lr_cut_folders]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

for root, dirs, files in os.walk(lr_raw_folders):
    for file in files:
        if "png" in file or "webp" in file:
            for scale in scales:
                base, ext = os.path.splitext(file)
                img_lr = cv2.imread(join(lr_raw_folders, file))
                img_lr_resize = cv2.resize(img_lr, (round(img_lr.shape[1]*scale/100), round(img_lr.shape[0]*scale/100)), interpolation=cv2.INTER_CUBIC)
                img_hr = cv2.imread(join(hr_raw_folders, file))
                img_hr_resize = cv2.resize(img_hr, (round(img_hr.shape[1]*scale/100), round(img_hr.shape[0]*scale/100)), interpolation=cv2.INTER_CUBIC)
                h, w = img_lr_resize.shape[:2]
                no_tile_h, no_tile_w = [math.ceil(d / tile_size) for d in [h, w]]
                for i in range(no_tile_h):
                    for j in range(no_tile_w):
                        y = int((h - tile_size) / (no_tile_h - 1) * i)
                        x = int((w - tile_size) / (no_tile_w - 1) * j)
                        
                        cut = img_lr_resize[y : y + tile_size, x : x + tile_size]
                        cv2.imwrite(join(lr_cut_folders, f"{scale}_{base}_{i}_{j}.png"), cut)
                        cut_hr = img_hr_resize[y*2 : y*2 + tile_size*2, x*2 : x*2 + tile_size*2]
                        cv2.imwrite(join(hr_cut_folders, f"{scale}_{base}_{i}_{j}.png"), cut_hr)

In [4]:
# resize HR and LR image and combine them to single folder
import cv2
import os
import math
from os.path import join
import shutil

scales = [0.5, 0.75, 1]


lr_raw = "E:/Dataset/amakano2/720p/"
hr_raw = "E:/Dataset/amakano2/1440p_escale/"
lr = f"R:/combine/lr/"
hr = f"R:/combine/hr/"

for folder_path in [hr, lr]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

for root, dirs, files in os.walk(lr_raw):
    for file in files:
        if "png" in file or "webp" in file:
            for scale in scales:
                for raw_path, resize_path in [
                    (lr_raw, lr),
                    (hr_raw, hr),
                ]:
                    img = cv2.imread(join(raw_path, file))
                    h, w = img.shape[:2]
                    img_resize = cv2.resize(
                        img,
                        (int(w * scale), int(h * scale)),
                        interpolation=cv2.INTER_CUBIC,
                    )
                    cv2.imwrite(join(resize_path, str(scale) + file), img_resize)